In [3]:
#Step 1. 필요한 모듈과 라이브러리를 import 합니다

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys

import math
import numpy  
import pandas as pd  
import xlwt

#Step 2. 필요한 정보를 입력 받습니다
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))

page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

f_name = input('3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fc_name = input('4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fx_name = input('5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( ) #검색 시작과 종료시간을 말해준다 
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/main/main.html')
time.sleep(2)

#Step 4. 검색창의 이름을 찾아서 검색어를 입력 후 검색을 진행합니다
driver.find_element_by_id("btnSearch").click()
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
driver.find_element_by_link_text("검색").click()

# 학습목표 1: 사용자가 입력한 건수와 실제 검색 건수를 비교 후 동기화하기
# Step 5. 사용자가 입력한 건수와 실제 검색 건수를 비교 후 동기화하기

cnt2=0

time.sleep(2)     # 페이지가 로딩 될 때까지 2초 기다립니다

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
search_cnt_1 = soup.find('div','total_check')
search_cnt_2 = search_cnt_1.find('strong').find('span').get_text() 
search_cnt_2 = search_cnt_2.replace("," , "")  

if cnt > int(search_cnt_2) :
    cnt2 = int(search_cnt_2)
else :
    cnt2 = cnt

real_page_cnt = math.ceil(cnt2/10)

print("\n")
print("="*80)
print("요청하신 검색 건수는 %s 건이었고" %cnt)
print("해당 키워드로 실제 검색된 글의 건수가 총 %s 건이라서" %search_cnt_2)
print("실제 검색 건수는 %s 건으로 재설정하고" %cnt2)
print("총 %s 페이지의 게시글 수집을 진행하겠습니다" %real_page_cnt)
print("="*80)
print("\n")

1.크롤링할 키워드는 무엇입니까?: 가을여행
2.크롤링 할 건수는 몇건입니까?: 10000
3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test3.txt
4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test3.csv
5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test3.xls


ValueError: invalid literal for int() with base 10: '1,020'

In [2]:
#학습 목표 2: 페이지 이동하기
#Step 6. 페이지 번호를 바꾸면서 실제 데이터 추출하기

no = 1            # 게시글 번호 카운트용 변수

no2 =[ ]          # 게시글 번호 저장용 리스트
contents2=[ ]     # 게시글 내용 저장용 리스트
tags2=[ ]         # 해쉬 태그 정보 저장용 리스트

#blog_list = soup.select('ul > li > dl') // ('ul>li>dl') 이렇게하면 안됨

for x in range(1,real_page_cnt+1) :  # 1페이지부터 반복 시작    
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    print("\n")    
    time.sleep(2)              #페이지가 로딩 될 때까지 2초 대기
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType flnon').find_all('div','area_txt')
    
    for i in content_list:        
        # 게시글 번호 입력
        no2.append(no)
        print('번호:',no)
    
        # 게시글의 내용 추출 및 입력
        contents = i.find('div','tit').get_text( )
        contents2.append(contents)
        print('내용:',contents.strip())
    
        # 학습목표 3: 예외 처리 사용하기
        try :
            tag = i.find('p','tag').get_text()
        except AttributeError :
            tag = '태그정보가 없습니다'
            tags2.append(tag)
            print('태그:',tag.strip())
            print("\n")       
        else :
            tags2.append(tag)
            print('태그:',tag.strip())
            print("\n")
        
        if no == cnt2 :
            break
    
        no += 1
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    x += 1
    
    if x == real_page_cnt+1 :   
        break

    if x == 6 :
        driver.find_element_by_link_text("""다음""").click()
    else :
        driver.find_element_by_link_text("""%s""" %x).click()     

#Step 7. 출력 결과를 표(데이터 프레임) 형태로 만들기

korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents2
korea['태그']=tags2

# 추출 결과를 txt 파일로 저장하기
f = open(f_name, 'a',encoding='UTF-8')
f.write(str(no2))
f.write(str(contents2))
f.write(str(tags2))
f.close( )

# 추출 결과를 csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 추출 결과를 엑셀 형태로 저장하기
korea.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %f_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

1.크롤링할 키워드는 무엇입니까?: 양산여행
2.크롤링 할 건수는 몇건입니까?: 20
3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\ys.txt
4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\ys.csv
5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\ys.xls


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17763 x86_64)
